In [5]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# data loading and pre-processing

import PiecewiseLinearSegmentation

x = PiecewiseLinearSegmentation.Bottomup(max_error=500)
n = []
f = open('DataSets/eth.csv', 'r')
for line in f:
    n.append(float(line))
data = []
for trend in x.transform(n):
    for val in trend:
        data.append(val)

import numpy as np
import torch
from torch.autograd import Variable

seq_length = 8
train_proportion = 0.8

def sliding_window(data):
    inputs = []
    outputs = []
    for i in range(0, len(data)-seq_length-1, 2):
        inputs.append(data[i:(i+seq_length)])
        outputs.append(data[i+seq_length:i+seq_length+2])
    return Variable(torch.Tensor(np.array(inputs))), Variable(torch.Tensor(np.array(outputs)))

total_data_input, total_data_output = sliding_window(data)
train_size = int(len(total_data_input)*train_proportion)

training_data_input = torch.narrow(total_data_input, 0, 0, train_size)
training_data_output = torch.narrow(total_data_output, 0, 0, train_size)

testing_data_input = torch.narrow(total_data_input, 0, train_size, len(total_data_input) - train_size)
testing_data_output = torch.narrow(total_data_output, 0, train_size, len(total_data_input) - train_size)

train = torch.utils.data.TensorDataset(training_data_input, training_data_output)
test = torch.utils.data.TensorDataset(testing_data_input, testing_data_output)

C:\Users\morga\anaconda3\lib\site-packages\PiecewiseLinearSegmentation\bottomup.py:27: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  ( p, residuals, rank, s ) = lstsq( X, y )


In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(8, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=8, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=2, bias=True)
)


In [27]:
# training

import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
for epoch in range(10):
    for data in train:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs.view(1,8))
        #print(outputs[0][1])
        loss = F.mse_loss(outputs[0], labels)
        loss.backward()
        optimizer.step()
    print(loss.item())

tensor(851.2440, grad_fn=<SelectBackward>)
tensor(814.3417, grad_fn=<SelectBackward>)
tensor(792.3444, grad_fn=<SelectBackward>)
tensor(768.7740, grad_fn=<SelectBackward>)
tensor(741.9407, grad_fn=<SelectBackward>)
tensor(715.5781, grad_fn=<SelectBackward>)
tensor(699.2860, grad_fn=<SelectBackward>)
tensor(678.0191, grad_fn=<SelectBackward>)
tensor(675.6572, grad_fn=<SelectBackward>)
tensor(670.5220, grad_fn=<SelectBackward>)
tensor(655.5043, grad_fn=<SelectBackward>)
tensor(641.0441, grad_fn=<SelectBackward>)
tensor(623.1633, grad_fn=<SelectBackward>)
tensor(605.7375, grad_fn=<SelectBackward>)
tensor(589.6045, grad_fn=<SelectBackward>)
tensor(565.8499, grad_fn=<SelectBackward>)
tensor(541.5262, grad_fn=<SelectBackward>)
tensor(514.0719, grad_fn=<SelectBackward>)
tensor(489.2371, grad_fn=<SelectBackward>)
tensor(473.9378, grad_fn=<SelectBackward>)
tensor(464.8756, grad_fn=<SelectBackward>)
tensor(455.6859, grad_fn=<SelectBackward>)
tensor(459.4561, grad_fn=<SelectBackward>)
tensor(459.

KeyboardInterrupt: 

In [10]:
# test

with torch.no_grad():
    for data in test:
        X, y = data
        output = net(X.view(-1,8))
        print(output)


tensor([[26624.1543,  2364.2966]])
tensor([[26602.5234,  2362.3772]])
tensor([[26617.6074,  2363.7219]])
tensor([[26634.1816,  2365.1885]])
tensor([[26623.0957,  2364.2021]])
tensor([[26665.7637,  2367.9934]])
tensor([[26710.8887,  2371.9819]])
tensor([[26648.8477,  2366.4568]])
tensor([[26644.6055,  2366.0935]])
tensor([[26694.5098,  2370.5129]])
tensor([[26660.8613,  2367.5120]])
tensor([[26640.9922,  2365.7537]])
tensor([[26670.9355,  2368.4104]])
tensor([[26671.1230,  2368.4177]])
tensor([[26640.6562,  2365.7112]])
tensor([[26630.9316,  2364.8528]])
tensor([[26611.5137,  2363.1301]])
tensor([[26595.9082,  2361.7559]])
tensor([[26623.5488,  2364.2195]])
tensor([[26648.2324,  2366.4099]])
tensor([[26635.6621,  2365.2920]])
tensor([[26628.9160,  2364.6990]])
tensor([[26637.9102,  2365.4995]])
tensor([[26638.5957,  2365.5618]])
tensor([[26654.6660,  2366.9915]])
tensor([[26662.3848,  2367.6719]])
tensor([[26639.7227,  2365.6587]])
tensor([[26612.5566,  2363.2520]])
tensor([[26605.5840,